## Seeding

In [1]:
import torch
import numpy as np
import random
import os
import sys

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

sys.path.append("..")
set_seed(42)

## Load Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("jtz18/skin-lesion")

c:\Users\chuanshaof\Projects\50.039-DL\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\chuanshaof\Projects\50.039-DL\venv\lib\site-packages\datasets\load.py:1461: FutureWarning: The repository for jtz18/skin-lesion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/jtz18/skin-lesion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


## Init Model

In [3]:
from src.segFormer2 import SegFormer
from src.unet import UNET
from src.unetr import UNETR

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

model = SegFormer(    
    in_channels=3,
    widths=[64, 128, 256, 512],
    depths=[3, 4, 6, 3],
    all_num_heads=[1, 2, 4, 8],
    patch_sizes=[7, 3, 3, 3],
    overlap_sizes=[4, 2, 2, 2],
    reduction_ratios=[8, 4, 2, 1],
    mlp_expansions=[4, 4, 4, 4],
    decoder_channels=256,
    scale_factors=[8, 4, 2, 1],
    num_classes=1
).to(device)

torch.Size([1, 1, 224, 224])


In [4]:
from src.utils.utils import *

_, _, test_loader = get_loaders(
    dataset,
    1000,
    None,
    None
)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
loss_fn = torch.nn.BCEWithLogitsLoss()

## Evaluation

In [5]:
start_epoch = 0

CHECKPOINT_FILENAME = "checkpoint.pth.tar"
checkpoint = torch.load(CHECKPOINT_FILENAME)
model, optimizer, start_epoch = load_checkpoint(checkpoint, model, optimizer)

check_accuracy(test_loader, model, device=device, mode="test")

FileNotFoundError: [Errno 2] No such file or directory: 'checkpoint.pth.tar'